In [ ]:
from estival.model import BayesianCompartmentalModel
from estival import targets as est

import nevergrad as ng
from estival.wrappers.nevergrad import optimize_model

import tbh.runner_tools as rt
from tbh.model import get_tb_model

from tbh.plotting import visualise_mle_params, plot_single_fit
from matplotlib import pyplot as plt


In [ ]:

def plot_age_spec_tbi_prev(derived_outputs, bcm):
    agegroups = ["5", "10", "15", "65"]
    
    model_vals = []
    targets = []

    # Collect model output (single trajectory) and observed target per age group
    for age in agegroups:
        output_name = f"measured_tbi_prevalenceXage_{age}_perc"
        year = 2024
        model_val = derived_outputs[output_name].loc[year]   # now just a scalar
        model_vals.append(model_val)

    # --- Plot ---
    fig, ax = plt.subplots(figsize=(8, 5))

    x = range(len(agegroups))

    # Modelled values (single run)
    ax.scatter(x, model_vals, color='blue', s=80, label='Modelled')

    # Observed target values

    # Labels and formatting
    ax.set_xticks(x)
    ax.set_xticklabels([
        f"{age}-{int(agegroups[i + 1]) - 1}" if i < (len(agegroups) - 1)
        else f"{age}+"
        for i, age in enumerate(agegroups)
    ])
    ax.set_xlabel("Age group (years)")
    ax.set_ylabel("measured_tbi_prevalence_perc")
    ax.set_title("Observed vs modelled TST positivity fraction by age group")
    ax.legend(loc='best')
    ax.grid(alpha=0.3)

    ax.set_ylim(bottom=0, top=45)

    plt.tight_layout()
    return fig



In [ ]:
params, priors, tv_params = rt.get_parameters_and_priors()
model = get_tb_model(rt.DEFAULT_MODEL_CONFIG, tv_params)

params['prev_se_cleared'] = 0.25
new_priors = [p for p in priors if p.name not in ["clearance_rate", "prev_se_cleared"]]

# Experiment one: no clearance

In [ ]:
params['clearance_rate'] = 0.  # fixing this parameter
bcm = BayesianCompartmentalModel(model, params, new_priors, rt.targets)

In [ ]:
opt_class = ng.optimizers.NGOpt
orunner = optimize_model(bcm, opt_class=opt_class)
rec = orunner.minimize(1000)
mle_params = rec.value[1]
res = bcm.run(mle_params)

In [ ]:
visualise_mle_params(bcm.priors, mle_params)

In [ ]:
plot_single_fit(bcm, mle_params)

In [ ]:
fig = plot_age_spec_tbi_prev(res.derived_outputs, bcm)

# Experiment two: with clearance

In [ ]:
params['clearance_rate'] = .5  # fixing this parameter
bcm = BayesianCompartmentalModel(model, params, new_priors, rt.targets)

In [ ]:
opt_class = ng.optimizers.NGOpt
orunner = optimize_model(bcm, opt_class=opt_class)
rec = orunner.minimize(10000)
mle_params = rec.value[1]
res = bcm.run(mle_params)

In [ ]:
visualise_mle_params(bcm.priors, mle_params)

In [ ]:
plot_single_fit(bcm, mle_params)

In [ ]:
fig = plot_age_spec_tbi_prev(res.derived_outputs, bcm)